In [1]:
import pandas as pd

data = pd.read_csv('data.tsv', sep = '\t', keep_default_na = False, error_bad_lines = False, dtype='unicode')

C:\Users\pesch\AppData\Local\Temp\ipykernel_14528\3451019758.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('data.tsv', sep = '\t', keep_default_na = False, error_bad_lines = False, dtype='unicode')


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
wordsToDel = stopwords.words('english')
data['abs'] = data['Abstract'].apply(word_tokenize)
data['abs'] = [word for word in data['abs'] if not word in wordsToDel]

In [ ]:
from nltk.stem import WordNetLemmatizer
def lemmatize(text):
  wnl = WordNetLemmatizer()
  return [wnl.lemmatize(w) for w in text]
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer('english', ignore_stopwords=False)
data['lem'] = data['abs'].apply(lemmatize)